In [9]:
import pandas as pd
import datetime as dt

csv_file = 'cleaned_online_retail_II_data.csv'
df = pd.read_csv(csv_file)

In [10]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
reference_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

In [11]:
rfm = df.groupby("Customer ID").agg(
    Recency=("InvoiceDate", lambda x: (reference_date - x.max()).days),
    Frequency=("Invoice", "nunique"),
    Monetary=("TotalPrice", "sum"),
    Country=("Country", "first")  # Ülke bilgisini ilk kayıttan al
)

rfm["Recency"] = rfm["Recency"].astype(int)

In [12]:
rfm.head()

,Recency,Frequency,Monetary,Country
Customer ID,,,,
12346.0,326,1,77183.60,United Kingdom
12347.0,2,7,4310.00,Iceland
12348.0,75,4,1797.24,Finland
12349.0,19,1,1757.55,Italy
12350.0,310,1,334.40,Norway


In [13]:
rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3, 4,5])

In [14]:
rfm["RFM_Score"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)

seg_map = {
    r'[1-2][1-2]': 'Hibernating',        # Düşük F, Düşük R -> Uyuyanlar
    r'[1-2][3-4]': 'At Risk',           # Yüksek F, Düşük R -> Risk Altında (Uzun süredir gelmemiş ama önceden sık geliyordu)
    r'[1-2]5': 'Cannot Lose Them',     # En Yüksek F, Düşük R -> Kaybedilmemesi Gerekenler
    r'3[1-2]': 'About To Sleep',       # Orta R, Düşük F -> Uyumak Üzere
    r'33': 'Need Attention',          # Orta R, Orta F -> İlgi Gerekenler
    r'[3-4][4-5]': 'Loyal Customers',     # Orta/Yüksek R, Yüksek F -> Sadık Müşteriler
    r'41': 'Promising',               # Yüksek R (Yeni), Düşük F -> Gelecek Vaat Edenler
    r'51': 'New Customers',           # En Yüksek R (Çok Yeni), Düşük F -> Yeni Müşteriler
    r'[4-5][2-3]': 'Potential Loyalists', # Yüksek R, Orta F -> Potansiyel Sadıklar
    r'[4-5][4-5]': 'Champions'            # En Yüksek R, En Yüksek F -> Şampiyonlar (En iyiler)
}
rfm["Segment_RF"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str)
rfm["Segment"] = rfm["Segment_RF"].replace(seg_map, regex=True)

In [15]:
rfm.head()

,Recency,Frequency,Monetary,Country,R_Score,F_Score,M_Score,RFM_Score,Segment_RF,Segment
Customer ID,,,,,,,,,,
12346.0,326,1,77183.60,United Kingdom,1,1,5,115,11,Hibernating
12347.0,2,7,4310.00,Iceland,5,5,5,555,55,Champions
12348.0,75,4,1797.24,Finland,2,4,4,244,24,At Risk
12349.0,19,1,1757.55,Italy,4,1,4,414,41,Promising
12350.0,310,1,334.40,Norway,1,1,2,112,11,Hibernating


In [16]:
rfm_output_file = "rfm_analysis_country_II_results.csv"
rfm.reset_index().to_csv(rfm_output_file, index=False)
print(f"\nRFM analizi sonuçları {rfm_output_file} dosyasına kaydedildi")


RFM analizi sonuçları rfm_analysis_country_II_results.csv dosyasına kaydedildi
